# MA Cross Strategy

This notebook implements an MA (Moving Average) cross strategy for trading. It includes data downloading, signal generation, backtesting, and parameter sensitivity analysis.

In [1]:
import logging
import os
import numpy as np
from app.utils import get_data
from tools.filter_portfolios import filter_portfolios
from tools.parameter_sensitivity_analysis import parameter_sensitivity_analysis

# Logging setup
log_dir = 'logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
logging.basicConfig(filename=os.path.join(log_dir, 'ema_cross.log'), level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

## Configuration

In [2]:
# Configuration
TICKER_1 = 'SPY'  # Ticker for X to USD exchange rate
TICKER_2 = 'BTC-USD'  # Ticker for Y to USD exchange rate
YEARS = 30  # Set timeframe in years for data
PERIOD = 'max' # Set time period for maximum data
USE_HOURLY = False  # Set to False for data
USE_SYNTHETIC = False  # Toggle between synthetic and original ticker
SHORT = False  # Set to True for short-only strategy, False for long-only strategy
USE_SMA = True  # Set to True to use SMAs, False to use EMAs
USE_GBM = False # Set to True to use Geometric Brownian Motion data

# Default Configuration
CONFIG_DEFAULT = {
    "PERIOD": 'max',
    "USE_HOURLY": False,
    "USE_SYNTHETIC": False,
    "TICKER_1": TICKER_1,
    "TICKER_2": TICKER_2,
    "SHORT": False,
    "USE_GBM": False,
    "BASE_DIR": 'C:/Projects/trading'
}

# Custom Configuration
CONFIG_CUSTOM = {
    "PERIOD": PERIOD,
    "USE_HOURLY": USE_HOURLY,
    "USE_SYNTHETIC": USE_SYNTHETIC,
    "TICKER_1": TICKER_1,
    "TICKER_2": TICKER_2,
    "SHORT": SHORT,
    "BASE_DIR": 'C:/Projects/trading'
}

CONFIG = CONFIG_CUSTOM

## Get Price Data

In [3]:
data = get_data(CONFIG)

data

[*********************100%***********************]  1 of 1 completed


Date,Open,High,Low,Close,Adj Close,Volume
datetime[ns],f64,f64,f64,f64,f64,i64
1993-01-29 00:00:00,43.96875,43.96875,43.75,43.9375,24.608622,1003200
1993-02-01 00:00:00,43.96875,44.25,43.96875,44.25,24.783653,480500
1993-02-02 00:00:00,44.21875,44.375,44.125,44.34375,24.836149,201300
1993-02-03 00:00:00,44.40625,44.84375,44.375,44.8125,25.098694,529400
1993-02-04 00:00:00,44.96875,45.09375,44.46875,45.0,25.203718,531500
…,…,…,…,…,…,…
2024-10-16 00:00:00,579.780029,582.830017,578.960022,582.299988,582.299988,30725400
2024-10-17 00:00:00,585.909973,586.119995,582.159973,582.349976,582.349976,34393700
2024-10-18 00:00:00,584.070007,585.390015,582.580017,584.590027,584.590027,37416800


## Analyse Parameter Sensitivity

In [4]:
short_windows = np.linspace(5, 50, 45, dtype=int)
long_windows = np.linspace(5, 50, 45, dtype=int)
portfolios = parameter_sensitivity_analysis(data, short_windows, long_windows, CONFIG)

portfolios

Analysis complete. Portfolios written to C:/Projects/trading\csv/ma_cross/SPY_parameter_portfolios_EMA.csv
Total rows in output: 990


Short Window,Long Window,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,Total Closed Trades,Total Open Trades,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
i64,i64,str,str,str,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64
41,44,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7652.180186,665.218019,1227.613103,100.0,258.191705,39.976111,"""2577 days 00:00:00""",49,48,1,1732.657704,45.833333,45.023562,-7.304961,13.522382,-3.288445,"""208 days 14:10:54.545454544""","""38 days 00:55:23.076923077""",3.131492,102.490052,0.716638,0.243697,1.130183,1.002223
42,43,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7652.180186,665.218019,1227.613103,100.0,258.191705,39.976111,"""2577 days 00:00:00""",49,48,1,1732.657704,45.833333,45.023562,-7.304961,13.522382,-3.288445,"""208 days 14:10:54.545454544""","""38 days 00:55:23.076923077""",3.131492,102.490052,0.716638,0.243697,1.130183,1.002223
37,48,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7603.165112,660.316511,1227.613103,100.0,258.876355,39.976111,"""2575 days 00:00:00""",50,49,1,1721.559383,46.938776,45.023562,-6.239333,12.691245,-3.210336,"""199 days 12:31:18.260869566""","""37 days 23:04:36.923076923""",3.222447,99.624607,0.715003,0.242892,1.129929,0.99976
39,46,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7601.029623,660.102962,1227.613103,100.0,257.799428,39.976111,"""2577 days 00:00:00""",50,49,1,1721.075852,46.938776,45.023562,-6.239333,12.739107,-3.247019,"""199 days 12:31:18.260869566""","""38 days 00:00:00""",3.211038,99.590893,0.714843,0.242856,1.129876,0.999467
41,47,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7584.443527,658.444353,1227.613103,100.0,239.283378,38.721552,"""2577 days 00:00:00""",46,45,1,1717.320317,46.666667,42.537178,-7.954504,14.145433,-3.613102,"""219 days 18:17:08.571428572""","""41 days 07:00:00""",3.089617,108.158294,0.712696,0.250442,1.129082,0.996631
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
6,9,"""0""","""7989""","""7990 days 00:00:00""",1000.0,692.461295,-30.75387,1227.613103,100.0,438.423759,71.957799,"""6669 days 00:00:00""",341,340,1,28.0998,33.235294,22.634286,-9.048947,3.645495,-1.908612,"""30 days 19:45:07.964601770""","""6 days 16:35:56.828193832""",0.884491,-0.987172,-0.054723,-0.023136,0.989778,-0.074125
5,7,"""0""","""7989""","""7990 days 00:00:00""",1000.0,690.707633,-30.929237,1227.613103,100.0,535.426528,74.006742,"""6488 days 00:00:00""",418,417,1,28.028637,34.532374,18.245116,-7.902754,3.171289,-1.750292,"""24 days 00:40:00""","""5 days 09:13:50.769230769""",0.893303,-0.808923,-0.054724,-0.022649,0.989687,-0.074391
5,10,"""0""","""7989""","""7990 days 00:00:00""",1000.0,685.705597,-31.42944,1227.613103,100.0,464.63568,70.666621,"""6669 days 00:00:00""",354,353,1,27.825656,33.711048,22.634286,-9.048947,3.561616,-1.905968,"""29 days 17:44:52.436974790""","""6 days 07:29:13.846153846""",0.886543,-0.969179,-0.058885,-0.024182,0.989051,-0.079714


## Filter Results

In [5]:
filtered_portfolios = filter_portfolios(portfolios, CONFIG)

filtered_portfolios

Analysis complete. Results written to C:/Projects/trading\csv/ma_cross/SPY_filtered_portfolios_EMA.csv
Total rows in output: 66


Metric Type,Short Window,Long Window,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,Total Closed Trades,Total Open Trades,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
str,i64,i64,str,str,str,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64
"""Most Total Return [%]""",41,44,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7652.180186,665.218019,1227.613103,100.0,258.191705,39.976111,"""2577 days 00:00:00""",49,48,1,1732.657704,45.833333,45.023562,-7.304961,13.522382,-3.288445,"""208 days 14:10:54.545454544""","""38 days 00:55:23.076923077""",3.131492,102.490052,0.716638,0.243697,1.130183,1.002223
"""Most Sharpe Ratio""",41,44,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7652.180186,665.218019,1227.613103,100.0,258.191705,39.976111,"""2577 days 00:00:00""",49,48,1,1732.657704,45.833333,45.023562,-7.304961,13.522382,-3.288445,"""208 days 14:10:54.545454544""","""38 days 00:55:23.076923077""",3.131492,102.490052,0.716638,0.243697,1.130183,1.002223
"""Most Omega Ratio""",41,44,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7652.180186,665.218019,1227.613103,100.0,258.191705,39.976111,"""2577 days 00:00:00""",49,48,1,1732.657704,45.833333,45.023562,-7.304961,13.522382,-3.288445,"""208 days 14:10:54.545454544""","""38 days 00:55:23.076923077""",3.131492,102.490052,0.716638,0.243697,1.130183,1.002223
"""Most Sortino Ratio""",41,44,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7652.180186,665.218019,1227.613103,100.0,258.191705,39.976111,"""2577 days 00:00:00""",49,48,1,1732.657704,45.833333,45.023562,-7.304961,13.522382,-3.288445,"""208 days 14:10:54.545454544""","""38 days 00:55:23.076923077""",3.131492,102.490052,0.716638,0.243697,1.130183,1.002223
"""Most Profit Factor""",35,48,"""0""","""7989""","""7990 days 00:00:00""",1000.0,7455.032707,645.503271,1227.613103,100.0,245.882594,38.444311,"""2633 days 00:00:00""",51,50,1,1688.018256,48.0,46.811565,-6.239333,12.241732,-3.344528,"""194 days 01:00:00""","""34 days 16:36:55.384615384""",3.27333,95.340289,0.709191,0.250006,1.129047,0.989378
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Least Expectancy""",5,6,"""0""","""7989""","""7990 days 00:00:00""",1000.0,500.442482,-49.955752,1227.613103,100.0,514.098102,74.639346,"""6488 days 00:00:00""",454,453,1,8.216185,34.216336,17.239497,-6.877064,3.042051,-1.752695,"""22 days 01:05:01.935483871""","""5 days 00:57:59.194630872""",0.834318,-1.120913,-0.160753,-0.041706,0.969902,-0.216995
"""Least Sharpe Ratio""",5,6,"""0""","""7989""","""7990 days 00:00:00""",1000.0,500.442482,-49.955752,1227.613103,100.0,514.098102,74.639346,"""6488 days 00:00:00""",454,453,1,8.216185,34.216336,17.239497,-6.877064,3.042051,-1.752695,"""22 days 01:05:01.935483871""","""5 days 00:57:59.194630872""",0.834318,-1.120913,-0.160753,-0.041706,0.969902,-0.216995
"""Least Calmar Ratio""",5,6,"""0""","""7989""","""7990 days 00:00:00""",1000.0,500.442482,-49.955752,1227.613103,100.0,514.098102,74.639346,"""6488 days 00:00:00""",454,453,1,8.216185,34.216336,17.239497,-6.877064,3.042051,-1.752695,"""22 days 01:05:01.935483871""","""5 days 00:57:59.194630872""",0.834318,-1.120913,-0.160753,-0.041706,0.969902,-0.216995


## Summarize Findings

Use an AI agent to consume the data and generate a comprehensive report with detailed reasoning.

## Visualize Findings